# Numerical PDE II: Spectral Methods

## Burgers' Equation

In the previous lecture, we analyzed finite difference methods for the
linear advection equation,
\begin{align}
  \frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0,
\end{align}
and studied their stability and accuracy using von Neumann analysis
and modified equations.

Now, we turn to a nonlinear extension: the Burgers' equation.
By replacing the constant wave speed $c$ with the solution variable
itself, the equation becomes
\begin{align}
  \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0.
\end{align}
This is the inviscid Burgers' equation, a canonical model for
nonlinear advection.

In [ ]:
# Parameters

l  = 1.0   # domain size
dt = 0.01  # time step

nx = 100   # number of spatial points
nt = 20    # number of time steps

In [ ]:
import numpy as np

X, dx = np.linspace(0, l, nx, endpoint=False, retstep=True)  # spatial grid
U0    = np.sin(2*np.pi * X)  # initial condition

In [ ]:
# Forward Time Centered Space (FTCS) scheme

def FTCS(U0, dx, dt, n):
    U = [U0]
    for _ in range(n):
        U0    = U[-1]
        sigma = U0 * dt / dx
        U1    = U0 - sigma * (np.roll(U0,-1) - np.roll(U0,1)) / 2
        U.append(U1)
    return np.array(U)

In [ ]:
UFTCS = FTCS(U0, dx, dt, nt) # numerical solution

In [ ]:
from matplotlib import pyplot as plt

plt.plot(X, U0,              label='Initial Condition')
plt.plot(X, UFTCS[10], '.-', label=f'$t$={10*dt}')
plt.plot(X, UFTCS[14], '.-', label=f'$t$={14*dt}')
plt.plot(X, UFTCS[18], '.-', label=f'$t$={18*dt}')
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

The Burgers' equation is important because:
* It demonstrates wave steepening: smooth initial profiles evolve into
  discontinuous shocks.
* It highlights the limitations of finite difference schemes:
  dispersion and oscillations arise near shocks.